## Importing Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    roc_curve,
    auc,
    mean_squared_error
)
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

## Training the data

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

def train_test_split_data(df, target_col, test_size=0.2, random_state=84):
    if target_col not in df.columns:
        raise ValueError(f"Target column '{target_col}' not found in the dataset.")
    
    X = df.drop(columns=[target_col])
    y = df[target_col]
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y if y.nunique() > 1 else None
    )
    
    # Apply Random OverSampling on training data
    over_sampler = RandomOverSampler(random_state=random_state)
    X_train_resampled, y_train_resampled = over_sampler.fit_resample(X_train, y_train)

    # Convert back to DataFrame for display
    X_train_df = pd.DataFrame(X_train_resampled, columns=X.columns)

    print("______MODEL INPUT FORMAT (1 sample)________")
    print(X_train_df.iloc[0].to_dict())  # As dictionary input to model
    # Optional: if you want it as a row DataFrame
    # print(X_train_df.iloc[[0]])

    return X_train_resampled, X_test, y_train_resampled, y_test


## Data Encoding

In [ ]:
def encode_data(df, cat_cols, encoder_path="encoders.pkl"):
    df_copy = df.copy()
    le_dict = {}
    
    for col in cat_cols:
        if col in df_copy.columns:
            le = LabelEncoder()
            df_copy[col] = le.fit_transform(df_copy[col].astype(str))
            le_dict[col] = le
        else:
            print(f"Warning: Column '{col}' not found in dataset, skipping encoding.")
    
    # Save the encoders to a separate file
    joblib.dump(le_dict, encoder_path)
    
    return df_copy

## Scaling the Data

In [ ]:
def preprocess_data(df, cat_cols=None, num_cols=None, encoder_path="encoders.pkl", scaler_path="scaler.pkl"):
    df_copy = df.copy()
    
    if cat_cols:
        df_copy = encode_data(df_copy, cat_cols, encoder_path)
    
    if num_cols:
        scaler = StandardScaler()
        df_copy[num_cols] = scaler.fit_transform(df_copy[num_cols])
        
        # Save the scaler to a separate file
        joblib.dump(scaler, scaler_path)
    
    return df_copy

##  Hyperparameter Fine Tuning

In [ ]:
def tune_hyperparameters(model, param_grid, X_train, y_train, method="grid"):
    
    if method == "grid":
        search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
    elif method == "random":
        search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=5, random_state=42, n_jobs=-1, verbose=1)
    else:
        raise ValueError("Method must be either 'grid' or 'random'.")
    
    search.fit(X_train, y_train)
    print(f"Best Parameters for {model.__class__.__name__}: {search.best_params_}")
    
    return search.best_estimator_

# Model Training and Evaluationg

In [ ]:
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def train_and_evaluate_models(X_train, y_train, X_test, y_test, model_save_path="best_model.pkl"):
    
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, class_weight="balanced"),
        'Decision Tree': DecisionTreeClassifier(random_state=42, class_weight="balanced"),
        'Random Forest': RandomForestClassifier(random_state=42, class_weight="balanced"),
        'Naive Bayes': GaussianNB(),
    }
    
    results = []
    trained_models = {}

    for model_name, model in models.items():
        if model_name == "Random Forest":
            param_grid = {'n_estimators': [100, 200, 400, 600], 'max_depth': [None, 10, 20, 40, 100], 'min_samples_split': [2, 5, 7, 10]}
            best_model = tune_hyperparameters(model, param_grid, X_train, y_train, method="random")
        elif model_name == "Decision Tree":
            param_grid = {'max_depth': [None, 10, 20, 40, 60, 100], 'min_samples_split': [2, 5, 7], 'criterion': ['gini', 'entropy']}
            best_model = tune_hyperparameters(model, param_grid, X_train, y_train, method="grid")
        else:
            best_model = model
        
        best_model.fit(X_train, y_train)
        trained_models[model_name] = best_model
        
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', zero_division=1)
        
        results.append({
            'Model': model_name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Identify the best model based on F1 Score
    best_model_row = results_df.loc[results_df['Accuracy'].idxmax()]
    best_model_name = best_model_row['Model']
    best_model = trained_models[best_model_name]
    
    # Save the best model
    try:
        joblib.dump(best_model, model_save_path)
        print(f"Best model ({best_model_name}) saved to {model_save_path}")
    except Exception as e:
        print(f"Error saving best model to {model_save_path}: {e}")
    
    return results_df, trained_models, best_model_name, best_model

## Plotting confusion matrix, ROC curve, and MSE graph

In [ ]:
def plot_graphs(models, X_test, y_test):
    """Plot model evaluation metrics attractively: Accuracy, Confusion Matrices, ROC Curve, and MSE."""
    
    model_names = list(models.keys())
    colors = ['#FF6F61', '#6B5B95', '#88B04B', '#F7CAC9']  # Custom colors

    # 1. Compute all evaluation metrics
    accuracies = []
    mse_values = []
    fpr_tpr_auc = {}  # Store ROC data  

    plt.figure(figsize=(12, 8))
    for i, (model_name, model) in enumerate(models.items()):
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else y_pred

        # Compute metrics
        accuracies.append(accuracy_score(y_test, y_pred))
        mse_values.append(mean_squared_error(y_test, y_pred))
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        auc_score = auc(fpr, tpr)
        fpr_tpr_auc[model_name] = (fpr, tpr, auc_score)

    # 2. Accuracy Bar Chart
    plt.figure(figsize=(10, 6))
    sns.barplot(x=model_names, y=accuracies, palette=colors)
    plt.title("Model Accuracy Comparison", fontsize=14, fontweight='bold')
    plt.ylabel("Accuracy Score")
    plt.ylim(0, 1)
    for i, acc in enumerate(accuracies):
        plt.text(i, acc + 0.02, f"{acc:.2f}", ha='center', fontsize=12)
    plt.show()

    # 3. ROC Curves in One Graph
    plt.figure(figsize=(10, 6))
    for i, (model_name, (fpr, tpr, auc_score)) in enumerate(fpr_tpr_auc.items()):
        plt.plot(fpr, tpr, label=f"{model_name} (AUC = {auc_score:.2f})", color=colors[i])
    
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.title("ROC Curve Comparison", fontsize=14, fontweight='bold')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right", fontsize=12)
    plt.show()

    # 4. Confusion Matrices (2x2 Grid)
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle("Confusion Matrices", fontsize=16, fontweight='bold')

    for i, (model_name, model) in enumerate(models.items()):
        if i >= 4:
            break
        y_pred = model.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        ax = axes[i // 2, i % 2]
        sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", cbar=False, ax=ax, annot_kws={"size": 16})
        ax.set_title(f"{model_name}", fontsize=12, fontweight='bold')
        ax.set_xlabel("Predicted", fontsize=10)
        ax.set_ylabel("True", fontsize=10)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    # 5. MSE Comparison Bar Chart
    plt.figure(figsize=(10, 6))
    sns.barplot(x=model_names, y=mse_values, palette=colors)
    plt.title("Mean Squared Error (MSE) Comparison", fontsize=14, fontweight='bold')
    plt.ylabel("MSE Value")
    for i, mse in enumerate(mse_values):
        plt.text(i, mse + 0.02, f"{mse:.2f}", ha='center', fontsize=12)
    plt.show()

## Performance Table

In [ ]:
def create_performance_table(results):
    
    print("Performance Metrics:")
    print(results.to_string(index=False))

## Reading the dataset

In [ ]:
df = pd.read_csv("cleaned_dataset.csv")

In [ ]:
value_counts = df['loan_status'].value_counts()


In [ ]:

def identify_column_types(df):

    numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
    categorical_cols = df.select_dtypes(exclude=['number']).columns.tolist()

    return {"0": numerical_cols, "1": categorical_cols}


In [ ]:
column_types = identify_column_types(df)
cat_cols = column_types['1']
num_cols = column_types['0']
target_col = "loan_status"


In [ ]:
df_processed = preprocess_data(df, cat_cols, num_cols)


In [ ]:
df_processed

In [ ]:
X_train, X_test, y_train, y_test = train_test_split_data(df_processed, target_col)


In [ ]:
results_df, trained_models, best_model_name, best_model = train_and_evaluate_models(X_train, y_train, X_test, y_test)

In [ ]:
print(results_df)


In [ ]:
plot_graphs(trained_models, X_test, y_test)


In [ ]:
create_performance_table(results_df)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_feature_importances(trained_models, feature_names, top_n=10):
    for model_name, model in trained_models.items():
        # Tree-based models
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            indices = np.argsort(importances)[-top_n:]

            plt.figure(figsize=(10, 6))
            plt.title(f'Top {top_n} Feature Importances - {model_name}')
            plt.barh(range(len(indices)), importances[indices], align='center')
            plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
            plt.xlabel('Importance Score')
            plt.tight_layout()
            plt.show()

        # Logistic Regression
        elif model_name == "Logistic Regression" and hasattr(model, "coef_"):
            coefs = model.coef_[0]
            indices = np.argsort(np.abs(coefs))[-top_n:]

            plt.figure(figsize=(10, 6))
            plt.title(f'Top {top_n} Feature Influences - {model_name}')
            plt.barh(range(top_n), coefs[indices], align='center')
            plt.yticks(range(top_n), [feature_names[i] for i in indices])
            plt.xlabel('Coefficient Value')
            plt.axvline(0, color='grey', linestyle='--')
            plt.tight_layout()
            plt.show()

        # Naive Bayes
        elif model_name == "Naive Bayes" and hasattr(model, "feature_log_prob_"):
            log_probs = model.feature_log_prob_
            mean_log_prob = np.mean(log_probs, axis=0)
            indices = np.argsort(mean_log_prob)[-top_n:]

            plt.figure(figsize=(10, 6))
            plt.title(f'Top {top_n} Feature Influences - {model_name}')
            plt.barh(range(top_n), mean_log_prob[indices], align='center')
            plt.yticks(range(top_n), [feature_names[i] for i in indices])
            plt.xlabel('Mean Log Probability')
            plt.tight_layout()
            plt.show()

        else:
            print(f"[Info] Feature importance/influence not available for model: {model_name}")


In [ ]:
# After training
results_df, trained_models = train_and_evaluate_models(X_train, y_train, X_test, y_test)

# Feature importance plot
plot_feature_importances(trained_models, feature_names=X_train.columns, top_n=10)
